In [1]:
# Rode o código se precisar instalar o snscrape
#%pip install git+https://github.com/JustAnotherArchivist/snscrape.git

# Rode esse código se precisar instalar o Pandas
# !pip install pandas

In [1]:
# Imports
import snscrape.modules.twitter as sntwitter
import pandas as pd
import datetime as dt


In [2]:
def ScrapTweets(max_tweets: int, search_string: str):
    # Definindo o máximo de tweets
    maxTweets = max_tweets

    # Criando a lista para salvar os dados
    tweets_list = []

    # Usando o TwitterSearchScraper para fazer o scrape dos dados data e salvar em uma lista
    for i, tweet in enumerate(
        sntwitter.TwitterSearchScraper(search_string).get_items()
    ):
        if i > maxTweets:
            break
        tweets_list.append(
            [
                tweet.id,
                tweet.date,
                tweet.rawContent,
                tweet.lang,
            ]
        )
    # Criando um dataframe com os dados
    tweets_df = pd.DataFrame(
        tweets_list,
        columns=[
            "TweetId",
            "DataHora",
            "Texto",
            "Idioma",
        ],
    )
    # Adicionando a coluna de data, mês e hora
    tweets_df["dia"] = tweets_df.DataHora.dt.day
    tweets_df["mes"] = tweets_df.DataHora.dt.month
    tweets_df["ano"] = tweets_df.DataHora.dt.year

    return tweets_df


In [38]:
# Na string de busca você pode usar ou omitir os marcadores from e until
# Exemplo:  'rockinrio from:anitta until:2020-07-31'

tweets_triste = ScrapTweets(50000, "#triste")
tweets_triste["emocao"] = "triste"

tweets_chateado = ScrapTweets(25000, "#chateado")
tweets_chateado["emocao"] = "chateado"

tweets_chateada = ScrapTweets(25000, "#chateada")
tweets_chateada["emocao"] = "chateado"

tweets_feliz = ScrapTweets(50000, "#feliz")
tweets_feliz["emocao"] = "feliz"

tweets_amor = ScrapTweets(50000, "#amor")
tweets_amor["emocao"] = "amor"

tweets_raiva = ScrapTweets(50000, "#raiva")
tweets_raiva["emocao"] = "raiva"

tweets_inveja = ScrapTweets(50000, "#inveja")
tweets_inveja["emocao"] = "inveja"

tweets_ironia = ScrapTweets(50000, "#ironia")
tweets_ironia["emocao"] = "ironia"

tweets = pd.concat(
    [
        tweets_triste,
        tweets_chateado,
        tweets_chateada,
        tweets_feliz,
        tweets_amor,
        tweets_raiva,
        tweets_inveja,
        tweets_ironia,
    ]
)

# Exportar dataframe para CSV
tweets.to_csv("text-query-tweets.csv", sep=",", index=False)


In [13]:
# tweets = pd.read_csv("text-query-tweets.csv", sep=",")
# tweets.head()
tweets = pd.read_csv("text-query-tweets.csv", sep=",")

/opt/miniconda3/envs/fast/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3135: DtypeWarning: Columns (0,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
#removendo duplicatas
tweets.drop_duplicates(subset ="TweetId", keep = False, inplace = True)

# Filtrando o idimoa para português
tweets = tweets[tweets['Idioma'] == 'pt']

# Removendo colunas desnecessárias
tweets.drop(['TweetId', 'DataHora', 'Idioma', 'dia', 'mes', 'ano'], axis=1, inplace=True)

# Removendo os Hashtags, links e usuários
tweets['Texto'] = tweets['Texto'].str.replace(r'#\S+', '', case=False)
tweets['Texto'] = tweets['Texto'].str.replace(r'@\S+', '', case=False)
tweets['Texto'] = tweets['Texto'].str.replace(r'http\S+', '', case=False)

# Removendo as marcações de paragrafos
tweets['Texto'] = tweets['Texto'].str.replace('\n', ' ', case=False)

tweets.reset_index(drop=True, inplace=True)

# tweets.to_csv("naive_bayes_dataset.csv", sep=",", index=False, encoding='utf-8-sig')
tweets.to_parquet("naive_bayes_dataset.parquet", index=False)


<ipython-input-14-f37810fb9b6f>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets['Texto'] = tweets['Texto'].str.replace(r'#\S+', '', case=False)
<ipython-input-14-f37810fb9b6f>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets['Texto'] = tweets['Texto'].str.replace(r'@\S+', '', case=False)
<ipython-input-14-f37810fb9b6f>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets['Texto'] = tweets['Texto'].str.replace(r'http\S+', '', case=False)


In [12]:
tweets.head()

,Texto,emocao
0,"Estou de fora,",triste
1,"Promessa não deu certo, partiu pra ameaça",triste
2,ver minha cidade dividida por causa do ego po...,triste
3,"tô precisando, to meio",triste
4,É engraçado quando você namora por muito tempo...,triste
